<h3>准备数据</h3>

<h4>读取数据</h4>

In [1]:
from time import localtime, strftime
import requests
import json
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Flatten, BatchNormalization
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import L1L2, L1, L2
from tensorflow.keras.callbacks import History, TensorBoard
from tensorflow.keras.backend import clear_session
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

2023-11-13 10:11:43.546008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 10:11:43.546035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 10:11:43.546054: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 10:11:43.550595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 10:11:44.103640: W tensorflow/compiler/

<h4>设置tensorflow</h4>

In [2]:
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

2023-11-13 10:11:46.671809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 10:11:46.696226: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 10:11:46.696328: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

<h3>处理数据</h3>

<h4>读取数据</h4>

In [9]:
data = pd.read_csv("../data/pic/adjustment.tsv", delimiter="\t", skipinitialspace=True)
data

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,7.4,0.048,1.19,1.47,0.23,0.54,0.27,5.75,0.65,0.71,...,0.07,0.62,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
1,3.7,NaN,1.87,0.18,0.94,0.70,0.07,5.20,0.73,0.59,...,0.91,0.89,0.99,1.11,0.13,0.18,1.33,0.86,1.22,1.30
2,1.2,0.046,1.97,1.17,0.67,0.89,0.60,2.10,0.97,0.14,...,0.80,0.69,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,7.2,0.053,1.17,2.13,0.82,0.18,0.17,5.30,0.27,0.46,...,0.37,0.37,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,1.8,0.021,1.89,0.42,0.20,0.92,0.80,1.60,0.20,0.70,...,0.50,0.68,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,5.4,0.069,1.77,0.99,0.39,0.54,0.64,4.95,0.70,0.30,...,0.23,0.89,0.28,0.46,0.03,0.73,0.13,0.82,0.02,0.22
4746,8.0,0.078,1.87,2.91,0.93,0.13,0.51,2.50,0.56,0.09,...,0.09,0.23,0.25,0.76,0.82,0.05,0.79,0.44,0.21,0.92
4747,6.9,0.051,1.45,0.48,0.17,0.33,0.13,3.10,0.56,0.99,...,0.06,0.69,0.60,0.59,0.54,0.95,0.97,0.67,0.73,0.33
4748,9.9,0.095,NaN,2.79,0.08,0.06,0.75,5.20,0.02,0.69,...,0.44,0.84,0.48,0.85,0.26,0.24,0.35,0.10,NaN,0.07


<h4>处理缺失值</h4>

In [10]:
data.dropna(axis=0, inplace=True)
data

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,7.4,0.048,1.19,1.47,0.23,0.54,0.27,5.75,0.65,0.71,...,0.07,0.62,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
2,1.2,0.046,1.97,1.17,0.67,0.89,0.60,2.10,0.97,0.14,...,0.80,0.69,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,7.2,0.053,1.17,2.13,0.82,0.18,0.17,5.30,0.27,0.46,...,0.37,0.37,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,1.8,0.021,1.89,0.42,0.20,0.92,0.80,1.60,0.20,0.70,...,0.50,0.68,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56
5,7.6,0.097,1.89,0.75,0.91,0.01,0.82,4.60,0.20,0.03,...,0.16,0.25,0.62,0.33,1.13,1.67,1.12,1.23,1.55,1.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744,9.9,0.059,1.79,2.73,0.83,0.12,0.59,4.45,0.72,0.39,...,0.21,0.43,0.83,0.45,0.51,0.05,0.53,0.20,0.76,0.24
4745,5.4,0.069,1.77,0.99,0.39,0.54,0.64,4.95,0.70,0.30,...,0.23,0.89,0.28,0.46,0.03,0.73,0.13,0.82,0.02,0.22
4746,8.0,0.078,1.87,2.91,0.93,0.13,0.51,2.50,0.56,0.09,...,0.09,0.23,0.25,0.76,0.82,0.05,0.79,0.44,0.21,0.92
4747,6.9,0.051,1.45,0.48,0.17,0.33,0.13,3.10,0.56,0.99,...,0.06,0.69,0.60,0.59,0.54,0.95,0.97,0.67,0.73,0.33


In [11]:
x = data.iloc[:, 0:18]
y = data.iloc[:, 18:]
x.head(5)
y.head(5)

,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
2,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56
5,0.62,0.33,1.13,1.67,1.12,1.23,1.55,1.22


<h4>划分验证集</h4>

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3488, 18)
(873, 18)
(3488, 8)
(873, 8)


<h3>搭建模型并训练</h3>

<h4>搭建模型</h4>

In [50]:
model = Sequential([
    BatchNormalization(name="bn_0", input_shape=(x.shape[1], )),
    Dense(units=128, activation=LeakyReLU(), kernel_regularizer=L2(0.2), activity_regularizer=L1(0.1), name="dense_1"),
    Dense(units=128, activation=LeakyReLU(), kernel_regularizer=L2(0.2), activity_regularizer=L1(0.1), name="dense_2"),
    Dense(units=128, activation=LeakyReLU(), kernel_regularizer=L2(0.2), activity_regularizer=L1(0.1), name="dense_3"),
    Dropout(0.2, name="dropout_1"),
    Dense(units=y.shape[1], activation=None, name="dense_8")
], name="adjustment")

model.compile(
    loss=Huber(),
    optimizer=Adam(learning_rate=1e-4),
    metrics=["mae"]
)

model.summary()

Model: "adjustment"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bn_0 (BatchNormalization)   (None, 18)                72        
                                                                 
 dense_1 (Dense)             (None, 128)               2432      
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 36560 (142.81 KB)
Trainable params: 36524 

<h4>训练模型</h4>

In [52]:
history: History = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=32,
    epochs=300,
    verbose=1,
    workers=-1,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(log_dir="../logs/" + strftime("%Y%m%d-%H%M%S", localtime()))
    ]
)

Epoch 1/300


109/109 [==============================] - 1s 8ms/step - loss: 0.1470 - mae: 0.4512 - val_loss: 0.1432 - val_mae: 0.4487
Epoch 2/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1446 - mae: 0.4512 - val_loss: 0.1419 - val_mae: 0.4487
Epoch 3/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1442 - mae: 0.4512 - val_loss: 0.1416 - val_mae: 0.4487
Epoch 4/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1439 - mae: 0.4512 - val_loss: 0.1415 - val_mae: 0.4487
Epoch 5/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1438 - mae: 0.4512 - val_loss: 0.1413 - val_mae: 0.4488
Epoch 6/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1437 - mae: 0.4512 - val_loss: 0.1412 - val_mae: 0.4488
Epoch 7/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1436 - mae: 0.4512 - val_loss: 0.1412 - val_mae: 0.4488
Epoch 8/300
109/109 [==============================] - 1s 8ms/step - loss: 0.1436

In [48]:
!tensorboard --logdir ../logs --bind_all

2023-11-11 15:50:10.374790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 15:50:10.374822: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 15:50:10.374839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 15:50:10.378980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 15:50:10.872645: W tensorflow/compiler/

In [41]:
save_model(model, "/home/wxy/Industrial-Algorithm-Competition/models/adjustment/2")     # save_model(model, "../models/adjustment/1")

INFO:tensorflow:Assets written to: /home/wxy/Industrial-Algorithm-Competition/models/adjustment/2/assets


INFO:tensorflow:Assets written to: /home/wxy/Industrial-Algorithm-Competition/models/adjustment/2/assets


<h3>模型部署</h3>

<h4>部署服务</h4>

In [20]:
!sudo docker run -t \
    -p 9989:9989 \
    -v "/home/wxy/Industrial-Algorithm-Competition/models/adjustment/2:/models/adjustment/2" \
    -e MODEL_NAME=adjustment \
    tensorflow/serving:2.14.0-gpu

[sudo] password for wxy: 


<h3>测试数据</h3>

<h4>读取数据并测试</h4>

In [42]:
data = x_test.values.tolist()
headers = {"content-type": "application/json"}

In [44]:
import subprocess

result = subprocess.run("echo 123|sudo -S docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' 2615fe3d76fd" ,shell=True, stdout=subprocess.PIPE,stderr=subprocess.PIPE)
print(result.stdout) # 输出为1

b'172.17.0.3\n'


In [45]:
response = requests.post(
    url="http://172.17.0.3:8501/v1/models/adjustment:predict",
    data=json.dumps({"instances": data}),
    headers=headers
)
if response.status_code != 200:
    raise ValueError(response.text)

In [46]:
import numpy as np
y_pred = np.array(json.loads(response.text)["predictions"])

In [47]:
print(mean_squared_error(y_test, y_pred))

0.2806500963324274
